# Tutorial for Robotic Modeling using Mujoco 

This tutorial covers the basics of the physics modeling with Mujoco. We will primarily focus on rigid-body systems that are prevalent in most robotic applications. 

## Principles of Physics Modeling
In order to simulate a physical system, we typically need to tell the simulator three things:
1. **System models**, often described as ODE/PDE or just functions. Often times, it is hard to derive analytically the system (differential) equations. Therefore, the most important functionality of a simulator is to allow the user to easily specify physical properties and automatically translate these properties into (differential) equations. For rigid body robotic systems, we need to tell the simulator:
   - **Muti-body system**: articulated rigid body system consists of multiple rigid bodies connected through joints. We need to define each body and describe its physical properties (e.g. pos, orientation, inertia matrix, etc), and how it is connected to adjacent bodies (through joints) 
   - **Environment/interaction**: gravity direction, ground, table, obstacle, wall, person, other robots, etc. How the robot interacts with the environments depends on many physical properties that can be modified, including friction, contact, collision mechanisms, 
   - **Actuators/sensors**: robot's motion is completely determined by each joint's motion. Joint's motion is determined by sensing and actuation strategies. The simulation is about the closed-loop behavior of the robot's motion and its interaction with the environment. 
2. **Physics Engine**: This part determines how to solve the (differential) equations associated with the specified robot-environment physical system. Parameters that can be modified include simulation time step, integrator (Euler, RK4, implicit, implicitfast), tolerance (to terminate iterative solver), constraint solver (PGS, CG, Newton), number of iterations (for constraint solver), etc. For Mujoco xml, these parameters are mostly specified through the \<option\> element. 
3. **Visualization**: This part determines how to visualize the robotic system scene. In Mujoco xml, some key visualization components are listed below:
   -  **light**: body/light will create a light, which moves with the body where it is defined. To create a fixed light, define it in the world body. The lights created here are in addition to the default headlight which is always defined and is adjusted via the visual element. The maximum number of lights that can be active simultaneously is 8, counting the headlight.
   -  **Camera**: body/⁠camera creates a camera, which moves with the body where it is defined. To create a fixed camera, define it in the world body. ca
   -  **geom**: body/⁠geom creates a geom, and attaches it rigidly to the body within which the geom is defined. Multiple geoms can be attached to the same body. At runtime they determine the appearance and collision properties of the body. 
   -  **Material**: asset/⁠material creates a material asset. It can be referenced from skins, geoms, sites and tendons to set their appearance. For rigid-body system, material is referenced from geom associated with a body. 
   - **Body vs Geom**: \<body\>: Represents a rigid entity with mass, inertia, and movable via joints. It does not have a shape until \<geom\> elements are attached to it. While \<geom\>: Defines the shape and size, used for collisions and visualization. It does not have mass or inertia, but can have density specified to infer mass. 



# XML Dictionary
This part summerizes key elements in a Mujoco XML model that is important for rigid-body robotic systems. Complete and detailed information can be found in the official website 
https://mujoco.readthedocs.io/en/latest/XMLreference.html



## Frame orientation:

The frame orientation is specified using at most one of these attributes. The quat attribute has a default value corresponding to the null rotation, while the others are initialized in the special undefined state. Thus if none of these attributes are specified by the user, the frame is not rotated.

- quat: real(4), “1 0 0 0”
  If the quaternion is known, this is the preferred was to specify the frame orientation because it does not involve conversions. Instead it is normalized to unit length and copied into mjModel during compilation. When a model is saved as MJCF, all frame orientations are expressed as quaternions using this attribute.

- axisangle: real(4), optional
  These are the quantities (x,y,z,a) mentioned above. The last number is the angle of rotation, in degrees or radians as specified by the angle attribute of compiler. The first three numbers determine a 3D vector which is the rotation axis. This vector is normalized to unit length during compilation, so the user can specify a vector of any non-zero length. Keep in mind that the rotation is right-handed; if the direction of the vector (x,y,z) is reversed this will result in the opposite rotation. Changing the sign of a can also be used to specify the opposite rotation.

- euler: real(3), optional
  Rotation angles around three coordinate axes. The sequence of axes around which these rotations are applied is determined by the eulerseq attribute of compiler and is the same for the entire model.

- xyaxes: real(6), optional
  The first 3 numbers are the X axis of the frame. The next 3 numbers are the Y axis of the frame, which is automatically made orthogonal to the X axis. The Z axis is then defined as the cross-product of the X and Y axes.

- zaxis: real(3), optional
  The Z axis of the frame. The compiler finds the minimal rotation that maps the vector (0,0,1) into the vector specified here. This determines the X and Y axes of the frame implicitly. This is useful for geoms with rotational symmetry around the Z axis, as well as lights – which are oriented along the Z axis of their frame.


## body
- need to specify: name (optional), pos, orientation (see frame orientation section), and important sub-element (geom, inertia, joint, etc)

### body/inertial:
- This element specifies the mass and inertial properties of the body. If this element is not included in a given body, the inertial properties are inferred from the geoms attached to the body. When a compiled MJCF model is saved, the XML writer saves the inertial properties explicitly using this element, even if they were inferred from geoms. The inertial frame is such that its center coincides with the center of mass of the body, and its axes coincide with the principal axes of inertia of the body. Thus the inertia matrix is diagonal in this frame.
- *pos*: position of inertial frame. 
- *orientation*: of the inertial frame
- *mass*: positive number required
- *diaginertia* (real(3)): diagonal entries of the inertial matrix;
- *fullinertia*: real(6): Full inertia matrix M. Since M is 3-by-3 and symmetric, it is specified using only 6 numbers in the following order: M(1,1), M(2,2), M(3,3), M(1,2), M(1,3), M(2,3). 

### body/joint
A joint creates motion degrees of freedom between the body where it is defined and the body’s parent. If no joints are defined, the body is welded to its parent. Joints cannot be defined in the world body. At runtime the positions and orientations of all joints defined in the model are stored in the vector mjData.qpos, in the order in which they appear in the kinematic tree. The linear and angular velocities are stored in the vector mjData.qvel. These two vectors have different dimensionality when free or ball joints are used, because such joints represent rotations as unit quaternions.
  - **name**, **class**: optional 
  - **type**: 
    - *free*: 6Dof joint. This joint type is only allowed in bodies that are children of the world body. Free joints do not have a position within the body frame. Instead the joint position is assumed to coincide with the center of the body frame. Thus at runtime the position and orientation data of the free joint correspond to the global position and orientation of the body frame. Free joints cannot have limits.
    - *slide*: a sliding or prismatic joint with one translational degree of freedom. Such joints are defined by a position and a sliding direction. For simulation purposes only the direction is needed; the joint position is used for rendering purposes.
    - *hinge*: The rotation takes place around a specified axis through a specified position. This is the most common type of joint and is therefore the default. 
  - **pos**: Position of the joint, specified in the frame of the body where the joint is defined. For free joints this attribute is ignored.
  - **axis**: This attribute specifies the axis of rotation for hinge joints and the direction of translation for slide joints.

### body/geom
- need to specify: name (optional), class (optional), pos, orientation (see frame orientation section), type, size (different meanings for different types), material (reference to materials defined in assets), rgba, among others. 
  
- **type**: (default: "sphere")
  - "box": The box type defines a box. Three size parameters are required, corresponding to the half-sizes of the box along the X, Y and Z axes of the geom’s frame. 
  - "cylinder": defines a cylinder, which requires two size parameters: the radius and half-height of the cylinder. The cylinder is oriented along the Z axis of the geom’s frame. 
  - "mesh": defines a mesh. The geom must reference the desired mesh asset with the mesh attribute. Note that mesh assets can also be referenced from other geom types, causing primitive shapes to be fitted; see below. The size is determined by the mesh asset and the geom size parameters are ignored.
- **fromto**: This attribute can only be used with capsule, box, cylinder and ellipsoid geoms. It provides an alternative specification of the geom length as well as the frame position and orientation. The six numbers are the 3D coordinates of one point followed by the 3D coordinates of another point. 
- **pos**: (x,y,z) position 
- orietation related: see frame orietation section
- **material**: 
- **rgba**: 


## asset
Assets are not in themselves model elements. Model elements can reference them, in which case the asset somehow changes the properties of the referencing element. One asset can be referenced by multiple model elements. Since the sole purpose of including an asset is to reference it, and referencing can only be done by name, every asset has a name (which may be inferred from a file name when applicable).

### asset/mesh
MuJoCo works with triangulated meshes. They can be loaded from binary STL files, OBJ files or MSH files. 

### asset/material
It can be referenced from skins, geoms, sites and tendons to set their appearance. Materials are useful for adjusting appearance properties beyond color.

# Python Examples

## Example 1: Geometric objects

In [6]:
import numpy as np
import mujoco
import mujoco.viewer as viewer

xmlModel = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" pos="0 0 0" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="blue_sphere" pos=".2 .2 .2" size=".1" rgba="0 0 1 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xmlModel)
data = mujoco.MjData(model)
viewer.launch(model,data)


If we modify the position of the box and sphere, they do not move. They are geometric objects only for visulization. In order for them to move, they need to be attached to a body with motion freedom specified by a joint. Before adding joint, we first use asset to specify material and texture, use \<default\> to change the default setting for the geom element

In [2]:
xml_ground = """
<mujoco>
  <asset>  
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="6 6" texuniform="true" reflectance=".2"/>
    <material name="wall" rgba='.5 .5 .5 1'/>
  </asset>
  <default>
    <geom type="box" size=".05 .05 .05" />
  </default>

  <worldbody>
    <light name="light" pos="-.2 0 1"/>
    <geom name="ground" type="plane" size=".5 .5 10" material="grid" friction=".1"/>
    <camera name="y" pos="-.1 -.6 .3" xyaxes="1 0 0 0 1 2"/>
    <geom name="red_box" pos="0 0 0.2" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="blue_sphere" pos=".2 .2 .2" size=".1" rgba="0 0 1 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml_ground)
data = mujoco.MjData(model)
viewer.launch(model,data)

## Example 2： Multi-body Robotic Systems

Now let's add a body and a joint to the world frame (worldbody). Simulate the physical motion caused by gravity. 

In [37]:
xml = """
<mujoco>
    <option gravity="0 0 -9.81"/>        
  <worldbody>
  <geom name="ground" type="plane" pos="0 0 -0.25" size="10 10 10"/>
    <light name="top" pos="0 0 1"/>
    <body name="box_and_sphere" euler="0 0 -30">
      <joint name="swing" type="hinge" axis="1 0 0" pos="-.2 -.2 -.2"/>
      
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
viewer.launch(model,data)

Now let's have a scene with ground (modeled as plane) and two boxes. The box can move freely in space, which is modeled as a "free" joint relative to the worldbody. You can modify the friction coefficients to see the resulting sliding motion. 

In [68]:
xml3 = """
<mujoco>
  <asset>  
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="6 6"
     texuniform="true" reflectance=".2"/>
     <material name="wall" rgba='.5 .5 .5 1'/>
  </asset>
  <default>
    <geom type="box" size=".05 .05 .05" />
    <joint type="free"/>
  </default>

  <worldbody>
    <light name="light" pos="-.2 0 1"/>
    <geom name="ground" type="plane" size="10 10 10" material="grid"
     zaxis="-.3 0 1" friction=".5"/>
    <camera name="y" pos="-.1 -.6 .3" xyaxes="1 0 0 0 1 2"/>
    <body pos="0 1 .3">
      <joint/>
      <geom friction="0.3"/>
    </body>
    <body pos="0 0 .3">
      <joint/>
      <geom friction="1"/>
    </body>
  </worldbody>

</mujoco>
"""

# load
model = mujoco.MjModel.from_xml_string(xml3)
data = mujoco.MjData(model)
viewer.launch(model,data)


Finally, let's have a multi-body robot (3R)

In [71]:
xml4="""<mujoco model="3R_robot">
    <compiler angle="radian"/>
    <asset>  
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
        rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
        <material name="grid" texture="grid" texrepeat="6 6"
         texuniform="true" reflectance=".2"/>
         <material name="wall" rgba='.5 .5 .5 1'/>
    </asset>
    <default>
        <joint type="hinge" axis="0 0 1" limited="true"/>
        <geom type="cylinder" size=".025 .1" />
    </default>

    <worldbody>
        <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
        <geom type="plane" size="1 1 0.1" material="grid"/>
        
        <body name="BaseLink" pos="0 0 0.1">  
            <geom/>    
            <body name="link1" pos="0.1 0 0.1" euler="0 1.57 0">
                <joint name="joint2" range="-1.57079 1.57079"/>
                <geom rgba=".6 .2 .2 1"/>
                <body name="link3" pos="0 0 0.2">
                    <joint name="joint3" range="-1.57079 1.57079" axis="1 0 0"/>
                    <geom rgba=".2 .6 1 1"/>
                    <site name="end_effector" pos="0 0 0.1" size="0.01"/>
                </body>
            </body>
        </body>

    </worldbody>
    <keyframe>
        <!-- Initial configuration -->
        <key name="initial_pose" qpos="1.57 1.57"/>        
        <!-- Bent configuration -->
        <key name="bent_pose" qpos="1.57079 0.78539"/>
    </keyframe>

</mujoco>"""
# load
model = mujoco.MjModel.from_xml_string(xml4)

data = mujoco.MjData(model)
mujoco.mj_resetDataKeyframe(model, data, 1)

viewer.launch(model,data)